In [79]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import pprint
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

In [80]:
modelID = "gpt-3.5-turbo"

https://data.world/promptcloud/amazon-product-dataset-2020

DESCRIPTION
This dataset contains a sample of 30K records. This dataset is the latest from amazon product details dated jan-mar 2020
SUMMARY
Context
This dataset was created by our in house teams at PromptCloud and [DataStock . This dataset contains 30K records in it. You can download the whole dataset ](https://app.datastock.shop/](https://www.promptcloud.com/)[here ?site_name=Amazon_USA_Product_Details).

Content
This dataset contains the following:

Total Records Count: 697053 
Domain Name: amazon.com 
Date Range: 01st Jan 2020 - 31st Jan 2020  
File Extension: CSV

Available Fields: Uniq Id, Product Name, Brand Name, Asin, Category, Upc Ean Code, List Price, Selling Price, Quantity, Model Number, About Product, Product Specification, Technical Details, Shipping Weight, Product Dimensions, Image, Variants, SKU, Product Url, Stock, Product Details, Dimensions, Color, Ingredients, Direction To Use, Is Amazon Seller, Size Quantity Variant, Product Description

In [81]:
filename_org = '../../data/marketing_sample_for_amazon_com.csv'
filename = '../../data/marketing_sample_for_amazon_com_sub.csv'
df_catalog = pd.read_csv(filename_org)
len(df_catalog)
df_catalog = df_catalog.sample(frac=.1).reset_index(drop=True)
len(df_catalog)
# df_catalog[~df_catalog['Product Description'].isnull()].head(2)
df_catalog.head(2)


,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,75cf1c44eefc3cb4cf7572db41d57196,Learning Resources Reversible Graph It! Mat,NaN,NaN,Toys & Games | Learning & Education | Counting...,NaN,NaN,$14.99,NaN,LER0215,...,https://www.amazon.com/Learning-Resources-Reve...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,03c9f00cd851983e7b5c75ecfe418d32,Flutter Friends Hummingbird Nectar,NaN,NaN,Toys & Games | Kids' Electronics | Electronic ...,NaN,NaN,$3.97,NaN,JPL77703,...,https://www.amazon.com/Flutter-Friends-JPL7770...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [82]:
df_catalog = df_catalog[['Product Name', 'Category', 'Model Number', 'Technical Details']]
len(df_catalog)
df_catalog
print(df_catalog.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product Name       1000 non-null   object
 1   Category           908 non-null    object
 2   Model Number       819 non-null    object
 3   Technical Details  925 non-null    object
dtypes: object(4)
memory usage: 31.4+ KB
None


In [83]:
# fill na with Not Available string
for col in df_catalog.columns:
    df_catalog[col] = df_catalog[col].fillna('Not Available')

df_catalog = df_catalog[0:100]
df_catalog.to_csv(filename, index=False)
df_catalog

,Product Name,Category,Model Number,Technical Details
0,Learning Resources Reversible Graph It! Mat,Toys & Games | Learning & Education | Counting...,LER0215,Get kids up and on their feet to work with thi...
1,Flutter Friends Hummingbird Nectar,Toys & Games | Kids' Electronics | Electronic ...,JPL77703,Go to your orders and start the return Select ...
2,Folkmanis French Bulldog Hand Puppet Plush,Toys & Games | Puppets & Puppet Theaters | Han...,3066,Go to your orders and start the return Select ...
3,C&D Visionary P-DSX-4740 DSX Rainbow Llama Pin...,Toys & Games | Party Supplies | Piñatas,P-DSX-4740,Go to your orders and start the return Select ...
4,MightySkins Skin Compatible with Razor Turbo J...,Sports & Outdoors | Outdoor Recreation | Skate...,RATUJE-Eagle Nebula,show up to 2 reviews by default Do You Want Yo...
...,...,...,...,...
95,The Young Scientists Club Science Art Fusion R...,Toys & Games | Learning & Education | Science ...,WH-925-1164,Go to your orders and start the return Select ...
96,Creative Converting 85590 2 Count Construction...,Toys & Games | Party Supplies | Party Favors,85590,They've been working all day; send all the wor...
97,"Alexanders Costumes Kids Knicker Pants, Black,...","Clothing, Shoes & Jewelry | Costumes & Accesso...",11-129/B,Go to your orders and start the return Select ...
98,"Roma Costume Deluxe Pirate Captain Boot Cover,...","Clothing, Shoes & Jewelry | Costumes & Accesso...",Not Available,Not Available


In [85]:
# from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders import DataFrameLoader, CSVLoader



# file = '../../data/trials_data.csv'
loader = CSVLoader(file_path=filename, encoding='utf-8')
# loader = DataFrameLoader(df_catalog, page_content_column="Technical Details")
docs = loader.load()
print(len(docs))
print(docs[0].page_content)

100
Product Name: Learning Resources Reversible Graph It! Mat
Category: Toys & Games | Learning & Education | Counting & Math Toys
Model Number: LER0215
Technical Details: Get kids up and on their feet to work with this dramatically oversized graph-printed mat! How many children are wearing blue socks today? White socks? Print socks? Graph it on this 4 x 12 grid/Venn Diagram double-sided vinyl mat. Make math come to life in your classroom! | Brand Name Learning Resources Item Weight 5.6 ounces Product Dimensions 0.5 x 8.5 x 13 inches Item model number LER0215 Material Type Vinyl Number of Items 1 Manufacturer Part Number LER0215 | 5.6 ounces (View shipping rates and policies)


In [72]:
docs

[Document(page_content="Go to your orders and start the return Select the ship method Ship it! | Go to your orders and start the return Select the ship method Ship it! | The Chicago Bears White Uniform NFL action figure set is pure NFL fun for Electric Football and more! Great for kid's room, office, fan cave, cake decoration, fantasy football, homegating, tailgating, and creative play. Electric Football action figures in 5 classic football poses also perform at your command on an Electric Football field! Includes 11 highly detailed one inch NFL team figures in authentic NFL team colors and logos. | 1.6 ounces (View shipping rates and policies)", metadata={'Product Name': 'Chicago Bears\xa0Away Jersey NFL Action Figure Set', 'Category': 'Sports & Outdoors | Fan Shop | Toys & Game Room | Action Figures & Toy Figurines', 'Model Number': '9-01-W'}),
 Document(page_content='show up to 2 reviews by default Give your Parrot Bebop Quadcopter Drone Quadcopter Drone a stylish look and protect i

In [63]:
from langchain.evaluation.qa import QAGenerateChain
from langchain_openai import ChatOpenAI

example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model=modelID))

In [108]:
[{"doc": t} for t in docs[:5]]

[{'doc': Document(page_content='Product Name: Learning Resources Reversible Graph It! Mat\nCategory: Toys & Games | Learning & Education | Counting & Math Toys\nModel Number: LER0215\nTechnical Details: Get kids up and on their feet to work with this dramatically oversized graph-printed mat! How many children are wearing blue socks today? White socks? Print socks? Graph it on this 4 x 12 grid/Venn Diagram double-sided vinyl mat. Make math come to life in your classroom! | Brand Name Learning Resources Item Weight 5.6 ounces Product Dimensions 0.5 x 8.5 x 13 inches Item model number LER0215 Material Type Vinyl Number of Items 1 Manufacturer Part Number LER0215 | 5.6 ounces (View shipping rates and policies)', metadata={'source': '../../data/marketing_sample_for_amazon_com_sub.csv', 'row': 0})},
 {'doc': Document(page_content="Product Name: Flutter Friends Hummingbird Nectar\nCategory: Toys & Games | Kids' Electronics | Electronic Pets\nModel Number: JPL77703\nTechnical Details: Go to yo

In [107]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in docs[:5]]
)

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[chain/start] [chain:QAGenerateChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:QAGenerateChain > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\npage_content='Product Name: Learning Resources Reversible Graph It! Mat\\nCategory: Toys & Games | Learning & Education | Counting & Math Toys\\nModel Number: LER0215\\nTechnical Details: Get kids up and on their feet to work with this dramatically oversized graph-printed mat! How many children are wearing blue socks today? White socks? Print socks? Graph it on this 4 x 12 grid/Venn Diagram double-sided vinyl mat. Make math co

In [109]:
new_examples[0]['qa_pairs']['query']
new_examples

[{'qa_pairs': {'query': 'What is the product name of the item described in the document?',
   'answer': 'The product name is Learning Resources Reversible Graph It! Mat.'}},
 {'qa_pairs': {'query': 'What are some of the features of the Flutter Friends Hummingbird Nectar toy?',
   'answer': 'Some features of the Flutter Friends Hummingbird Nectar toy include the ability to interact with kids and other birds, over 40 realistic hummingbird sounds that can be unlocked by petting the head or back, soft shimmery wings, different sounds and tweets activated by tapping the head multiple times, and the option to help the Flutter friend learn to fly by tapping their back twice.'}},
 {'qa_pairs': {'query': 'What are some distinctive features of the Folkmanis French Bulldog hand puppet?',
   'answer': 'The Folkmanis French Bulldog hand puppet is distinctively featured with bat-shaped ears and a wrinkly face, making it the epitome of cuteness.'}},
 {'qa_pairs': {'query': 'What is the product name a

In [98]:
nprint(docs[4].page_content)

('Product Name: MightySkins Skin Compatible with Razor Turbo Jetts Electric '
 'Heel Wheels - Eagle Nebula | Protective, Durable, and Unique Vinyl wrap '
 'Cover | Easy to Apply, Remove, and Change Styles | Made in The USA\n'
 'Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & '
 'Scooters | Scooters & Equipment | Accessories\n'
 'Model Number: RATUJE-Eagle Nebula\n'
 'Technical Details: show up to 2 reviews by default Do You Want Your Razor '
 'Turbo Jets Electric Heel Wheels To Look Different Than The Rest? You’re in '
 'the right place because we’ve got exactly what you’re looking for! This '
 'Eagle Nebula skin is the perfect way to show off your style! Or with '
 'hundreds of other Mighty Skins designs, you can be sure to find one that '
 'you’ll love, and that will show off your unique style! Do You Want To '
 'Protect Your Razor Turbo Jets Electric Heel Wheels? With Mighty Skins your '
 'Turbo Jets Electric Heel Wheels is protected from scratches, dings, d

In [88]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embed_model_id = 'sentence-transformers/all-mpnet-base-v2'
embedding = HuggingFaceEmbeddings(model_name=embed_model_id)

# Directly creates a vectorstore in memory and loads the documents into it
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding = embedding
).from_loaders([loader])

d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [89]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA


# index.query("What are the three pieces included in the Disney Frozen 2 Giftable Sleepover Set?")

llm = ChatOpenAI(temperature = 0.0, model=modelID)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [110]:
new_examples

[{'qa_pairs': {'query': 'What is the product name of the item described in the document?',
   'answer': 'The product name is Learning Resources Reversible Graph It! Mat.'}},
 {'qa_pairs': {'query': 'What are some of the features of the Flutter Friends Hummingbird Nectar toy?',
   'answer': 'Some features of the Flutter Friends Hummingbird Nectar toy include the ability to interact with kids and other birds, over 40 realistic hummingbird sounds that can be unlocked by petting the head or back, soft shimmery wings, different sounds and tweets activated by tapping the head multiple times, and the option to help the Flutter friend learn to fly by tapping their back twice.'}},
 {'qa_pairs': {'query': 'What are some distinctive features of the Folkmanis French Bulldog hand puppet?',
   'answer': 'The Folkmanis French Bulldog hand puppet is distinctively featured with bat-shaped ears and a wrinkly face, making it the epitome of cuteness.'}},
 {'qa_pairs': {'query': 'What is the product name a

## Manual Evaluation

In [117]:
import langchain
langchain.debug = True
i_example = 4
answer = qa.run(new_examples[i_example]['qa_pairs']['query'])


[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What are some key features of the MightySkins Skin Compatible with Razor Turbo Jetts Electric Heel Wheels - Eagle Nebula?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What are some key features of the MightySkins Skin Compatible with Razor Turbo Jetts Electric Heel Wheels - Eagle Nebula?",
  "context": "Product Name: MightySkins Skin Compatible with Razor Turbo Jetts Electric Heel Wheels - Eagle Nebula | Protective, Durable, and Unique Vinyl wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA\nCategory: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Accessories\nModel Number: RATUJE-Eagle Nebula<<<<>>>>>Technical Details: show up to 2 reviews by default Do

In [118]:
print(answer)

Some key features of the MightySkins Skin Compatible with Razor Turbo Jetts Electric Heel Wheels - Eagle Nebula are:

1. **Protective:** The skin protects your Razor Turbo Jetts Electric Heel Wheels from scratches, dings, dust, fingertips, and everyday wear-and-tear.
   
2. **Durable:** Made with durable vinyl material that helps in protecting your device.
   
3. **Unique Design:** The Eagle Nebula design allows you to showcase your style and stand out from the crowd.
   
4. **Easy Application:** The skin is easy to apply and remove without leaving any sticky residue.
   
5. **Made in the USA:** The skin is proudly made in the USA, ensuring quality and reliability.


In [116]:
nprint(new_examples[i_example].get('qa_pairs')['answer'])

('Some key features of the MightySkins Skin Compatible with Razor Turbo Jetts '
 'Electric Heel Wheels - Eagle Nebula include the ability to protect the Turbo '
 'Jets Electric Heel Wheels from scratches, dings, dust, and everyday '
 'wear-and-tear, the ease of application and removal without leaving sticky '
 'residue, a vinyl decal sticker with a matte finish, ultra-thin and '
 'ultra-durable design, stain resistance, and a wide variety of unique designs '
 'to choose from.')


In [106]:
docs[:5]

[Document(page_content='Product Name: Learning Resources Reversible Graph It! Mat\nCategory: Toys & Games | Learning & Education | Counting & Math Toys\nModel Number: LER0215\nTechnical Details: Get kids up and on their feet to work with this dramatically oversized graph-printed mat! How many children are wearing blue socks today? White socks? Print socks? Graph it on this 4 x 12 grid/Venn Diagram double-sided vinyl mat. Make math come to life in your classroom! | Brand Name Learning Resources Item Weight 5.6 ounces Product Dimensions 0.5 x 8.5 x 13 inches Item model number LER0215 Material Type Vinyl Number of Items 1 Manufacturer Part Number LER0215 | 5.6 ounces (View shipping rates and policies)', metadata={'source': '../../data/marketing_sample_for_amazon_com_sub.csv', 'row': 0}),
 Document(page_content="Product Name: Flutter Friends Hummingbird Nectar\nCategory: Toys & Games | Kids' Electronics | Electronic Pets\nModel Number: JPL77703\nTechnical Details: Go to your orders and sta

In [105]:
print(docs[4].page_content)

Product Name: MightySkins Skin Compatible with Razor Turbo Jetts Electric Heel Wheels - Eagle Nebula | Protective, Durable, and Unique Vinyl wrap Cover | Easy to Apply, Remove, and Change Styles | Made in The USA
Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Accessories
Model Number: RATUJE-Eagle Nebula
Technical Details: show up to 2 reviews by default Do You Want Your Razor Turbo Jets Electric Heel Wheels To Look Different Than The Rest? You’re in the right place because we’ve got exactly what you’re looking for! This Eagle Nebula skin is the perfect way to show off your style! Or with hundreds of other Mighty Skins designs, you can be sure to find one that you’ll love, and that will show off your unique style! Do You Want To Protect Your Razor Turbo Jets Electric Heel Wheels? With Mighty Skins your Turbo Jets Electric Heel Wheels is protected from scratches, dings, dust, fingertips, and the wear-and-tear of everyday use! C

## LLM assisted evaluation

In [127]:
examples = []
for q in new_examples:
    examples.append(
        q.get('qa_pairs')
    )
examples   

[{'query': 'What is the product name of the item described in the document?',
  'answer': 'The product name is Learning Resources Reversible Graph It! Mat.'},
 {'query': 'What are some of the features of the Flutter Friends Hummingbird Nectar toy?',
  'answer': 'Some features of the Flutter Friends Hummingbird Nectar toy include the ability to interact with kids and other birds, over 40 realistic hummingbird sounds that can be unlocked by petting the head or back, soft shimmery wings, different sounds and tweets activated by tapping the head multiple times, and the option to help the Flutter friend learn to fly by tapping their back twice.'},
 {'query': 'What are some distinctive features of the Folkmanis French Bulldog hand puppet?',
  'answer': 'The Folkmanis French Bulldog hand puppet is distinctively featured with bat-shaped ears and a wrinkly face, making it the epitome of cuteness.'},
 {'query': 'What is the product name and category of the item described in the document?',
  'an

In [128]:
from langchain.evaluation.qa import QAEvalChain
# llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)
predictions = qa.apply(examples)
graded_outputs = eval_chain.evaluate(examples, predictions)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the product name of the item described in the document?",
  "answer": "The product name is Learning Resources Reversible Graph It! Mat."
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the product name of the item described in the document?",
  "context": "Product Name: JoJo Siwa Three Bow Bundle (Amazon Exclusive)\nCategory: Clothing, Shoes & Jewelry | Costumes & Accessories | Kids & Baby | Girls\nModel Number: Not Available<<<<>>>>>Product Name: Triple Eight Undercover Snow Knee Pads for Snowboarding and Skiing\nCategory: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Protective Gear | Knee Pads\nModel Number: 604352 90022-24<<<<>>>>>Product Name: Sun-Staches Costume S

In [135]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    nprint("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: What is the product name of the item described in the document?
'Real Answer: The product name is Learning Resources Reversible Graph It! Mat.'
Predicted Answer: The product name of the item described in the document is "d-c-fix 96448 Spring Chapel/Tulia Window Film, 17" x 78" Roll-4 Pk."
Predicted Grade: INCORRECT

Example 1:
Question: What are some of the features of the Flutter Friends Hummingbird Nectar toy?
('Real Answer: Some features of the Flutter Friends Hummingbird Nectar toy '
 'include the ability to interact with kids and other birds, over 40 realistic '
 'hummingbird sounds that can be unlocked by petting the head or back, soft '
 'shimmery wings, different sounds and tweets activated by tapping the head '
 'multiple times, and the option to help the Flutter friend learn to fly by '
 'tapping their back twice.')
Predicted Answer: Some features of the Flutter Friends Hummingbird Nectar toy include:
- Birds that interact with kids and each other
- Over 

In [133]:
graded_outputs

[{'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'}]

In [131]:
predictions[3]

{'query': 'What is the product name and category of the item described in the document?',
 'answer': 'The product name is C&D Visionary P-DSX-4740 DSX Rainbow Llama Pinata Patch, and the category is Toys & Games | Party Supplies | Piñatas.',
 'result': 'The product name is "Delta Children Deluxe Book & Toy Organizer, Disney Mickey Mouse" and the category is Baby Products | Nursery | Furniture | Storage & Organization | Toy Chests & Organizers.'}